<a href="https://colab.research.google.com/github/bhanu07x/Text-classification-using-ML/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification using ML

In [2]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-

In [1]:
import gensim
import pandas as pd
import numpy as np

In [2]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
datapath='/content/drive/MyDrive/IMDB Dataset.csv'

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import string
import nltk.corpus
import sklearn
import keras

In [6]:
df=pd.read_csv(datapath)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [25]:
df.shape


(50000, 2)

In [7]:
df=df.iloc[:10000]
df.shape

(10000, 2)

In [28]:
df.isnull().sum()

,0
review,0
sentiment,0


In [8]:
df.duplicated().sum()

17

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.duplicated().sum()

0

In [11]:
import re
def remove_tags(raw_text):
  clean=re.sub(r'<.*?>',"",raw_text)
  return clean

In [12]:
df["review"]=df["review"].apply(remove_tags)

In [13]:
df["review"]=df["review"].apply(lambda x:x.lower())

In [14]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
li=stopwords.words('english')
def remove_stopwords(text):
  new_text=[]
  for word in text.split():
    if word not in li:
      new_text.append(word)
  return " ".join(new_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
df["review"]=df["review"].apply(remove_stopwords)

In [17]:
x=df.iloc[:,0:1]
y=df['sentiment']

In [20]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer


In [33]:
cv=CountVectorizer()
x_train_bow=cv.fit_transform(x_train['review']).toarray()
x_test_bow=cv.transform(x_test['review']).toarray()

In [34]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [38]:
y_pred=gnb.predict(x_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6324486730095142

In [39]:
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]])

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train_bow,y_train)

RandomForestClassifier()

In [41]:
accuracy_score(y_test,rf.predict(x_test_bow))

0.8497746619929895

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_train_tfidf=vectorizer.fit_transform(x_train['review']).toarray()
x_test_tfidf=vectorizer.transform(x_test['review']).toarray()

In [44]:
rf=RandomForestClassifier()
rf.fit(x_train_tfidf,y_train)
y_pred=rf.predict(x_test_tfidf)

In [45]:
accuracy_score(y_test,y_pred)

0.8507761642463696

In [48]:
from gensim.models import Word2Vec

# Train Word2Vec
w2v_model = Word2Vec(sentences=df["review"], vector_size=100, window=5, min_count=1, workers=4)

In [49]:
import numpy as np

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # Handle empty reviews
    return np.mean(vectors, axis=0)

# Convert tokens to averaged embeddings
x_vectors = np.array([get_avg_vector(tokens, w2v_model) for tokens in df["review"]])

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x_vectors, y, test_size=0.2, random_state=1)

In [54]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.6074111166750125